In [1]:
from quchem.Ansatz_Generator_Functions import *

In [ ]:
from numpy import kron
import numpy as np
from functools import reduce
zero = np.array([[1], [0]])
one = np.array([[0], [1]])
STATE=[one,one,zero,zero]
reduce(kron, STATE)

In [ ]:
HF_reference_ket= ansatz_obj.Get_JW_HF_state().reshape([len(ansatz_obj.Get_JW_HF_state()), 1])

https://arxiv.org/pdf/1808.10402.pdf

$$U=e^{-i \theta Y_{0} X_{1} X_{2} X_{3}}$$

In [ ]:
from openfermion.ops import QubitOperator
from openfermion.transforms import get_sparse_operator
UCCSD_QubitOp = QubitOperator('Y0 X1 X2 X3', -1j)

UCCSD_operator = get_sparse_operator(UCCSD_QubitOp)

In [ ]:
pauliDict={'X':np.array([[0,1],[1,0]]),
          'Y':np.array([[0,-1j],[1j,0]]),
          'Z':np.array([[1,0],[0,-1]]),
          'I': np.eye(2)}

In [ ]:
list_of_ops = [pauliDict[term[0]] for term in 'Y0 X1 X2 X3'.split(' ')]
matrix = reduce(kron, list_of_ops) *-1j #NOTE NEED TO MULTIPLY BY THIS!

np.array_equal(UCCSD_operator.todense(), matrix)

In [ ]:
import scipy

theta=np.pi/10
ansatz_state_ket = scipy.sparse.linalg.expm_multiply(theta*UCCSD_operator, HF_reference_ket)
ansatz_state_ket

In [ ]:
# |Ψ H2〉= 0.9939|0011〉−0.1106|1100〉 === ground state!
# |0011〉 = |3〉
# |1100〉= |12〉

#coef = [0., 0., 0., 0.1106, 0., 0., 0., 0., 0., 0., 0., 0., 0.9939, 0., 0., 0.]

$$\theta = \frac{\pi}{2}$$

$$U=e^{-i \frac{\pi}{2} Y_{0} X_{1} X_{2} X_{3}}$$

$$U | \psi_{HF}\rangle = | \psi_{UCCSD}\rangle$$

In order to perfrom the following ciruict:

$$U = e^{-i \theta X_{0} Z_{1} Y_{2} X_{3}}$$

overall we get:

$\begin{aligned} U|1100\rangle &=\left(\cos (\theta) I-i \sin (\theta)  Y_{0}X_{1}X_{2} X_{3} \right)|1100\rangle \\ &=\cos (\theta)|1100\rangle-\sin (\theta)|0011\rangle \end{aligned}$

to do this in Q.C the following circuit is employed:


In [ ]:
import os
parent_dir = os.path.abspath('..')
# os.listdir(parent_dir) # lists directory
picture_folder = os.path.join(parent_dir, 'pictures')
# os.listdir(picture_folder) # lists directory
pic = os.path.join(picture_folder, 'exponentiated_pauliword.png')


from IPython.display import Image
Image(pic)

In [ ]:
#checking equivalence!

theta= np.pi/10

qubitOP = QubitOperator('Y0 X1 X2 X3', -1j)
matrix_op = get_sparse_operator(qubitOP)
expon_matrix = scipy.sparse.linalg.expm(theta*matrix_op)


full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))


np.isclose(np.around(UCCSD_circ.unitary(qubit_order=cirq.LineQubit.range(4)), 3), np.around(expon_matrix.todense(), 3))


## overall function

In [ ]:
from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *

def H2_ansatz(theta):
    
    HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
    
#     HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
#     full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('X0 X1 X2 Y3', -1j), theta)
    
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
circuit_form = H2_ansatz(np.pi/10)    
circuit_form

In [ ]:
measure_circ_obj = Measure_PauliWord(QubitOperator('Y0 X1 X2 X3', -1j))
measure_circ_cirq = cirq.Circuit(cirq.decompose_once((measure_circ_obj(*cirq.LineQubit.range(measure_circ_obj.num_qubits())))))

full_circ = cirq.Circuit([*circuit_form.all_operations(),*measure_circ_cirq.all_operations()])

num_shots=1000

simulator = cirq.Simulator()
raw_result = simulator.run(full_circ, repetitions=num_shots)
hist_result = raw_result.histogram(key='0,1,2,3')
hist_result

In [ ]:
# simulator = cirq.Simulator()
# result = simulator.simulate(circuit_form, qubit_order=circuit_form.all_qubits())
# state = np.around(result.final_state, 5)
# state.reshape([len(state),1])

circuit_form = H2_ansatz(np.pi/10)
from quchem.Simulating_Quantum_Circuit import Get_wavefunction
print('state')
print('')
Get_wavefunction(circuit_form)

In [ ]:
from openfermion.ops import QubitOperator
from openfermion.transforms import get_sparse_operator
UCCSD_QubitOp = QubitOperator('Y0 X1 X2 X3', -1j)

UCCSD_operator = get_sparse_operator(UCCSD_QubitOp)